In [3]:
import numpy as np

# package for 3d visualization
from itkwidgets import view                              
from aicssegmentation.core.visual import seg_fluo_side_by_side,  single_fluorescent_view, segmentation_quick_view
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [16, 12]

# package for io 
from aicsimageio import AICSImage
from aicsimageio.writers import OmeTiffWriter

# function for core algorithm
from aicssegmentation.core.vessel import filament_3d_wrapper
from aicssegmentation.core.pre_processing_utils import intensity_normalization, edge_preserving_smoothing_3d, image_smoothing_gaussian_3d
from skimage.morphology import remove_small_objects  
from tifffile import imsave

import pandas as pd

In [83]:
filename = "//allen/aics/microscopy/PRODUCTION/PIPELINE_8_1/5500000650_DD_3-01/aligned_split/5500000650_DD_3-01_AcquisitionBlock7_pt7_Scene-27_aligned.ome.tiff"
reader = AICSImage(filename) 
IMG = reader.data.astype(np.float32)

print(IMG.shape)

Attempted file (//allen/aics/microscopy/PRODUCTION/PIPELINE_8_1/5500000650_DD_3-01/aligned_split/5500000650_DD_3-01_AcquisitionBlock7_pt7_Scene-27_aligned.ome.tiff) load with reader: aicsimageio.readers.bfio_reader.OmeTiledTiffReader failed with error: No module named 'bfio'
The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).


(1, 4, 40, 1200, 1800)


In [84]:
#####################
## PARAMETER ##
structure_channel = 1
#####################

struct_img0 = IMG[0,structure_channel,:,:,:].copy()
# view(single_fluorescent_view(struct_img0))

<h3> Pre-Processing <h3>

In [85]:
################################
## PARAMETERS for this step ##
intensity_scaling_param = [0,50] #current: [0,60] v2: [0,60]
################################
# intensity normalization
struct_img = intensity_normalization(struct_img0, scaling_param=intensity_scaling_param)

# smoothing with edge preserving smoothing 
structure_img_smooth = edge_preserving_smoothing_3d(struct_img)

#view(single_fluorescent_view(structure_img_smooth))

In [86]:
# if contrast looks off: 
from aicssegmentation.core.pre_processing_utils import suggest_normalization_param
suggest_normalization_param(struct_img0)

mean intensity of the stack: 101.7747573852539
the standard deviation of intensity of the stack: 4.5995378494262695
0.9999 percentile of the stack intensity is: 217.0
minimum intensity of the stack: 100.4016342163086
maximum intensity of the stack: 366.76800537109375
suggested upper range is 25.5, which is 219.06297254562378
suggested lower range is 0.0, which is 101.7747573852539
So, suggested parameter for normalization is [0.0, 25.5]
To further enhance the contrast: You may increase the first value (may loss some dim parts), or decrease the second value(may loss some texture in super bright regions)
To slightly reduce the contrast: You may decrease the first value, or increase the second value


<h3> Core Algorithm <h3>

In [87]:
################################
## PARAMETERS for this step ## [[scale_1, cutoff_1], [scale_2, cutoff_2]]
f3_param = [[2,0.22],[1,0.017]] # original: [[2,0.2],[1,0.015]] current: [[2,0.22],[1,0.017]]
################################

bw = filament_3d_wrapper(structure_img_smooth, f3_param)

<h3> Post-Processing <h3>

In [88]:
################################
## PARAMETERS for this step ##
minArea = 4 # original: 4
################################

seg = remove_small_objects(bw>0, min_size=minArea, connectivity=1, in_place=False) #connectivity = original: 1

In [89]:
# save the file
out=seg.astype(np.uint8)
out[out>0]=255
imsave("myh10-50ms-b7-sc27v2.tiff", out)
#imsave("myh10-sc04-p23-b04.tiff", out)